In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
logits

tensor([[0.0773, 0.0988, 0.0254, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0638,
         0.1061]], device='cuda:0', grad_fn=<ReluBackward0>)

In [7]:
pred_prob = nn.Softmax(dim=1)(logits)
pred_prob

tensor([[0.1040, 0.1063, 0.0987, 0.0963, 0.0963, 0.0963, 0.0963, 0.0963, 0.1026,
         0.1070]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [9]:
y_pred = pred_prob.argmax(1)
y_pred

tensor([9], device='cuda:0')

In [11]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [15]:
print(f"Before ReLU: \n{hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: \n{hidden1}")

Before ReLU: 
tensor([[0.0000, 0.0000, 0.5178, 0.0000, 0.0000, 0.0000, 0.0000, 0.2425, 0.1112,
         0.0016, 0.1629, 0.0000, 0.3936, 0.1417, 0.0000, 0.0506, 0.0000, 0.3435,
         0.2514, 0.1812],
        [0.0000, 0.2904, 0.4408, 0.0000, 0.0000, 0.0000, 0.0000, 0.4558, 0.0000,
         0.3254, 0.1170, 0.0151, 0.4396, 0.0147, 0.0000, 0.2395, 0.0452, 0.5805,
         0.3111, 0.0000],
        [0.0000, 0.1322, 0.3438, 0.0000, 0.0000, 0.0000, 0.0000, 0.3535, 0.2384,
         0.5229, 0.3461, 0.0096, 0.5250, 0.0920, 0.0000, 0.0998, 0.0274, 0.2297,
         0.2314, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: 
tensor([[0.0000, 0.0000, 0.5178, 0.0000, 0.0000, 0.0000, 0.0000, 0.2425, 0.1112,
         0.0016, 0.1629, 0.0000, 0.3936, 0.1417, 0.0000, 0.0506, 0.0000, 0.3435,
         0.2514, 0.1812],
        [0.0000, 0.2904, 0.4408, 0.0000, 0.0000, 0.0000, 0.0000, 0.4558, 0.0000,
         0.3254, 0.1170, 0.0151, 0.4396, 0.0147, 0.0000, 0.2395, 0.0452, 0.5805,
         0.3111, 0.0000],
     

In [17]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits.shape)

torch.Size([3, 10])


In [18]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab.shape)

torch.Size([3, 10])


In [19]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0232,  0.0004, -0.0018,  ..., -0.0188, -0.0048, -0.0278],
        [-0.0051,  0.0199, -0.0269,  ..., -0.0337,  0.0200, -0.0197]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0177, -0.0343], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0269,  0.0439,  0.0333,  ...,  0.0300,  0.0315,  0.0436],
        [ 0.0433,  0.0397, -0.0439,  ..., -0.0287,  0.0390, -0.0072]],
       device='cuda